In [1]:
import sys
sys.path.append('..')
from model import Model
import torch
from PIL import Image
import numpy as np

In [2]:
model = Model()
model.load_state_dict(torch.load("../model_0.27.pth"))

<All keys matched successfully>

In [3]:
import torch

# Create a 4D tensor
tensor_4d = torch.empty(3, 3, 3, 3)

tensor_4d.dim()

4

In [4]:
print(model)

Model(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [5]:
# Load an example image
image_path = "../data/CityScapes/leftImg8bit/train/aachen/aachen_000001_000019_leftImg8bit.png"
mask_path = "../data/CityScapes/gtFine/train/aachen/aachen_000001_000019_gtFine_labelIds.png"
image = Image.open(image_path)
mask = Image.open(mask_path)

# Show the image
image.show()
mask.show()

In [6]:
from process_data import train_preprocess, val_preprocess

# Preprocess the image
image_t, mask_t = train_preprocess(image, mask)
image_v, mask_v = val_preprocess(image, mask)

# Check shapes
print(image_t.shape, mask_t.shape)
print(image_t.dtype, mask_t.dtype)
print(image_v.shape, mask_v.shape)
print(image_v.dtype, mask_v.dtype)
print(image_t.min(), image_t.max(), mask_t.min(), mask_t.max())
print(image_v.min(), image_v.max(), mask_v.min(), mask_v.max())

# Show the preprocessed image
image_s = image_t.permute(1, 2, 0)
image_s = image_s.numpy()
image_s = (image_s * 255).astype(np.uint8)
image_s = Image.fromarray(image_s)
image_s.show()

# Show the preprocessed mask
print(mask_t)


torch.Size([3, 520, 1040]) torch.Size([1, 520, 1040])
torch.float32 torch.uint8
torch.Size([3, 520, 1040]) torch.Size([1, 520, 1040])
torch.float32 torch.uint8
tensor(0.) tensor(1.) tensor(0, dtype=torch.uint8) tensor(26, dtype=torch.uint8)
tensor(-1.9295) tensor(2.6400) tensor(0, dtype=torch.uint8) tensor(26, dtype=torch.uint8)
tensor([[[3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3],
         ...,
         [3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3],
         [3, 3, 3,  ..., 3, 3, 3]]], dtype=torch.uint8)


In [ ]:
# Make a prediction
with torch.no_grad():
    prediction = model(image_t.unsqueeze(0))

In [ ]:
print(prediction.shape)
print(prediction)

In [ ]:
import torch.nn.functional as F
prediction_smax = F.softmax(prediction, dim=1)
print(prediction_smax.shape)
print(prediction_smax)

In [ ]:
prediction_argmax = torch.argmax(prediction_smax, dim=1)

print(prediction_argmax.shape)
print(prediction_argmax)
print(np.unique(prediction_argmax))

In [ ]:
from process_data import postprocess

# Postprocess the prediction
prediction = postprocess(prediction)

# Check size of prediction numpy array
print(prediction.shape)
print(np.unique(prediction))

In [ ]:
# Turn the prediction into an image
prediction_s = prediction
prediction_image = Image.fromarray(prediction_s)
prediction_image.show()